In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import ast
import pickle
import os

from Migros.helper import one_hot, one_hot_one
from collections import Counter
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
similarity_matrix = pickle.load(open( "Migros/recommender_data.p", "rb" ))
possible_ids = pickle.load(open( "possible_ids.p", "rb" ))

In [3]:
def predict(user_purchases):
    """
    This currently cannot handle  a variety of situations
        - if it is a new customer with no purchases
        - if the customer has made a purchase that is not in the similarity matrix
        - if the customer has purchased all items in the similarity matrix
    """
    
    if len(user_purchases) == 0:
        pop_dict = pickle.load(open('Migros/popular_items.p', 'rb'))
        return pop_dict
    
    user_rating_vector = one_hot_one(user_purchases, possible_ids)
    score = user_rating_vector.dot(similarity_matrix).transpose().div(pd.DataFrame(similarity_matrix.sum(axis=1)))
    
    df = score.drop(user_purchases).reset_index()
    print(df)
    df.columns = ['product_id', 'score']
    payload = df.sort_values('score', ascending = False).reset_index(drop=True).to_dict()
    
    return payload
    


In [9]:
possible_ids

[7881, 10834, 61603, 62040, 63288, 125830]

In [18]:
user_purchases = [63288,10834]

In [20]:
payload = predict(user_purchases)

    index         0
0    7881  0.347666
1   61603  0.308912
2   62040  0.328264
3  125830  0.371785


In [16]:
payload

{'product_id': {0: 10834, 1: 63288, 2: 125830, 3: 7881, 4: 61603, 5: 62040},
 'score': {0: 0.82, 1: 0.67, 2: 0.67, 3: 0.64, 4: 0.53, 5: 0.49}}